# 一个稍微复杂些的线性回归模型

<table class="mge-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="#"><img src="../../assets/logo/megengine-32.png" />在天元官网查看</a>
  </td>
  <td>
    <a target="_blank" href="#"><img src="../../assets/logo/megvii-32.svg" />在 MegStudio 中运行</a>
  </td>
  <td>
    <a target="_blank" href="#"><img src="../../assets/logo/github-32.svg" />在 GitHub 上查看源代码</a>
  </td>
  <td>
    <a href="#"><img src="../../assets/logo/download-32.svg" />下载当前教程文件</a>
  </td>
</table>

在 [上一个教程](beginner.ipynb) 中，我们学习了如何去训练一个简单的线性回归模型 $y = w * x + b$, 接下来我们将稍微升级一下难度：

- 我们对单个样本数据的表示形式将不再是简单的标量 $x$, 而是升级为多维的向量 $\boldsymbol {x}$ 表示；
- 我们将接触到更多不同形式的梯度下降策略，从而接触一个新的超参数 `batch_size`;
- 我们将尝试将数据集封装成 MegEngine 支持的 `Dataset` 类，方便在 `Dataloader` 中进行各种预处理操作；
- 同时，我们的前向计算过程将变成矩阵运算形式，能够帮助你更好地理解向量化实现的优点，以及由框架完成求梯度操作的好处；
- 在这个过程中，一些遗留问题会得到解答，同时我们将接触到一些新的机器学习概念。

本教程的话题将围绕着**“数据”（Data）**进行，希望你能够有所收获～

请先运行下面的代码，检验你的环境中是否已经安装好 MegEngine（[访问官网安装教程](https://megengine.org.cn/install)）：

In [1]:
import megengine

print(megengine.__version__)

1.1.0


接下来，我们将对将要使用的数据集 [波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) 进行一定的了解。

## 原始数据集的获取和分析

获取到真实的数据集原始文件后，往往需要做大量的数据格式处理工作，变成易于计算机理解的形式，才能被各种框架和库使用。

但目前我们的重心不在此处，使用 Python 机器学习库 [`scikit-learn`](https://scikit-learn.org/stable/index.html)，可以快速地获得波士顿房价数据集:

In [2]:
import numpy as np
from sklearn.datasets import load_boston

# 获取 boston 房价数据集，需要安装有 scikit-learn 这个库
boston = load_boston()

# 查看 boston 对象内部的组成结构
print(boston.keys())

# 查看数据集描述，内容比较多，因此这里注释掉了
# print(boston.DESCR)

# 查看 data 和 target 信息，这里的 target 可以理解成 label
print(boston.data.shape, boston.target.shape)  

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
(506, 13) (506,)


我们已经认识过了数据 `data` 和标签 `label`（标签有时也叫目标 `target` ），那么 `feature` 是什么？

我们在描述一个事物的时候，通常会寻找其属性（Attribute）或者说特征（Feature）：
- 比如我们描述一个人的长相，会说这个人的鼻子如何、眼睛如何、额头如何等等
- 又比如在游戏角色的属性经常有生命值、魔法值、攻击力、防御力等属性
- 类比到一些支持自定义角色的游戏，这些特征就变成许多可量化和调整的数据

实际上，你并不需要在意波士顿房价数据集中选用了哪些特征，我们在本教程中关注的更多是“特征维度变多”这一情况。

为了方便后续的交流，我们需要引入一些数学符号来描述它们，不用害怕，理解起来非常简单：
- 波士顿房价数据集的样本容量为 506，记为 $n$; （Number）
- 每个样本的特征有 13 个维度，包括住宅平均房间数、城镇师生比例等等，记为 $d$. （Dimensionality）
- 单个的样本可以用一个行向量  $\boldsymbol {x}=(x_{1}, x_{2}, \ldots , x_{d})$ 来表示，称为“特征向量”，里面的每个元素对应着该样本的某一维特征；
- 数据集 `data` 由 $n$ 行特征向量组成，每个特征向量有 $d$ 维度特征，因此整个数据集可以用一个形状为 $(n, d)$ 的数据矩阵 $X$ 来表示；
- 标签 `label` 的每个元素是一个标量值，记录着房价值，因此它本身是一个含有 $n$ 个元素的向量 $\boldsymbol {y}$.

$$
X = \begin{bmatrix}
- \boldsymbol {x}_1 - \\
- \boldsymbol {x}_2 - \\
\vdots \\
- \boldsymbol {x}_n - 
\end{bmatrix}
= \begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,d} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,d} \\
\vdots  &         &        & \vdots \\
x_{n,1} & x_{n,2} & \cdots & x_{n,d} 
\end{bmatrix}
\quad
\boldsymbol {y} =  (y_{1}, y_{2}, \ldots, y_{d})
$$

其中 $x_{i,j}$ 表示第 $i$ 个样本 $\boldsymbol {x_i}$ 的第 $j$ 维特征，$y_i$ 为样本 $\boldsymbol {x_i}$ 对应的标签。

- 不同的人会对数学符号的使用有着不同的约定，一定需要搞清楚这些符号的定义，或者在交流时采用比较通用的定义方式
- 为了简洁美观，我们这里用粗体表示向量 $\boldsymbol {x}$; 而在手写时粗细体并不容易辨别，所以板书时常用上标箭头来表示向量 $\vec {x}$.

In [3]:
X, y = boston.data, boston.target
num, dim = X.shape

print(X[0], y[0]) # 取出第一个样本，查看对应的特征向量和标签

[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00] 24.0


### 数据集的划分

- 对于常见的机器学习任务，用作基准（Benchmark）的数据集通常会被分为训练集（Training dataset）和测试集（Test dataset）两部分；
  - 我们利用训练集的数据，通过优化损失函数的形式将模型训练好，在测试集上对模型进行测试，根据选用的度量指标（Metrics）评价模型性能
  - 测试集的数据不能参与模型训练，因为我们希望测试集数据代表着将来会被用于预测的真实数据，它们现在是“不可见的”，当作标签未知
- 训练模型时为了调整和选取合适的超参数，通常还会在训练集的基础上划分出验证集（Validation dataset）或者说开发集（Develop dataset）
  - 为了避免在短时间内引入过多密集的新知识，我们目前将不会进行验证集和开发集的代码实践和讨论

数据集该如何划分是一个值得讨论与研究的话题，它是数据预处理的一个环节，目前我们只需要对它有一个基本概念，通过不断实践来加深理解。

对于波士顿房价数据集，我们可以将 506 张样本划分为 500 张训练样本，6 张测试样本：

In [4]:
mask = 500  # 这个值是随意设计的，不用想太多背后的原因

train_dataset = X[:mask]
test_dataset = X[mask:num]
train_label = y[:mask]
test_label = y[mask:num]

print(len(train_dataset), len(train_label))
print(len(test_dataset), len(test_label))

500 500
6 6


## 模型定义、训练和测试

对于单个的样本 $\boldsymbol {x}$, 想要得到输出 $y$, 即尝试找到映射关系 $f: \boldsymbol {x} \in \mathbb {R}^{d} \mapsto y$, 同样可以建立线性模型：

$$
\begin{aligned}
y &= f(\boldsymbol {x}) = \boldsymbol {w} \cdot \boldsymbol {x} + b \\
& = (w_{1}, w_{2}, \ldots, w_{13}) \cdot (x_{1}, x_{2}, \ldots, x_{13}) + b\\
& = w_{1} x_{1} + w_{2} x_{2} + \ldots + w_{13} x_{13} + b
\end{aligned}
$$

**注意在表示不同的乘积（Product）时，不仅使用的符号有所不同，数学概念和编程代码之间又有所区别：** 
- 上个教程中用的星乘 $*$ 符号指的是元素间相乘（有时用 $\odot$ 表示），也适用于标量乘法，编程对应于 `np.multiply()` 方法
- 对于点积我们用 $\cdot$ 符号表示，编程对应于 `np.dot()` 方法，在作用于两个向量时它等同于内积 `np.inner()` 方法
  - 根据传入参数的形状，`np.dot()` 的行为也可以等同于矩阵乘法，对应于 `numpy.matmul()` 方法或 `@` 运算符
  - 严谨地说，点积或者数量积只是内积的一种特例，**但我们在编程时，应该习惯先查询文档中的说明，并进行简单验证**

比如样本 $\boldsymbol {x}$ 和参数 $\boldsymbol {w}$ 都是 13 维的向量，二者的点积结果是一个标量：

In [5]:
w = X[0]
x = np.random.random((13,))
print(x.shape, w.shape) # 一维张量，即为向量

p = np.dot(x, w)        # Dot Product
print(p.shape)          # 零维张量，即为标量

(13,) (13,)
()


我们之前已经实现过类似的训练过程，现在一起来看下面这份新实现的代码：

In [6]:
epochs = 5
lr = 1e-6

w = np.zeros((13,))  
b = np.zeros(())

data = train_dataset
label = train_label
n = len(data)

def linear_model(x):
    return np.dot(x, w) + b   # 已经不再是 x * w 

for epoch in range(epochs):
    
    loss = 0
    
    sum_grad_w = 0
    sum_grad_b = 0
    
    for i in range(n):
        
        pred = linear_model(data[i])
        loss += (pred - label[i]) ** 2 
        
        sum_grad_w += 2 * (pred - label[i]) * data[i]
        sum_grad_b += 2 * (pred - label[i])
    
    grad_w = sum_grad_w / n
    grad_b = sum_grad_b / n
    
    w = w - lr * grad_w
    b = b - lr * grad_b
    
    loss = loss / n
    print("epoch = {}, loss = {:.3f}".format(epoch, loss))

epoch = 0, loss = 594.442
epoch = 1, loss = 197.173
epoch = 2, loss = 138.077
epoch = 3, loss = 126.176
epoch = 4, loss = 121.148


现在，我们可以使用训练得到的 `w` 和 `b` 在测试数据上进行评估，这里我们可以选择使用平均绝对误差（Mean Absolute Error）作为度量：


$$
\ell(y_{pred}, y_{real})= \frac{1}{n }\sum_{i=1}^{n}\left | \hat{y}_{i}-{y}_{i}\right |
$$

In [7]:
data = test_dataset
label = test_label
n = len(data)
    
loss = 0

for i in range(n):
    pred = linear_model(data[i])   
    loss += np.abs(pred - label[i])
    print("Pred = {:.3f}, Real = {:.3f}".format(pred, label[i]))

loss = loss / n

print("Average loss = {:.3f}".format(loss))

Pred = 21.814, Real = 16.800
Pred = 18.942, Real = 22.400
Pred = 19.130, Real = 20.600
Pred = 19.191, Real = 23.900
Pred = 19.075, Real = 22.000
Pred = 19.148, Real = 11.900
Average loss = 4.137


### 不同的梯度下降形式

我们可以发现：上面的代码中，我们每训练一个完整的 `epoch`, 将根据所有样本的梯度进行一次参数更新。

这种通过对所有的样本的计算来求解梯度的方法叫做**批梯度下降法(Batch Gradient Descent)**.

当碰到样本容量特别大的情况时，可能会导致无法一次性将所有数据给读入内存，遇到内存用尽（Out of memory，OOM）的情况。

这时你可能会想：“其实我们完全可以在每个样本经过前向传播计算损失、反向传播计算得到梯度后时，就立即对参数进行更新呀！”

这种思路叫做**随机梯度下降（Stochastic Gradient Descent，常缩写成 SGD)**，动手改写后，整体代码实现如下：

In [8]:
epochs = 5
lr = 1e-6

w = np.zeros((13,))  
b = np.zeros(())

data = train_dataset
label = train_label
n = len(data)

def linear_model(x):
    return np.dot(x, w) + b

for epoch in range(epochs):
    
    loss = 0
    
    for i in range(n):
        
        pred = linear_model(data[i])     
        loss += (pred - label[i]) ** 2    
        
        w -= lr * 2 * (pred - label[i]) * data[i]
        b -= lr * (pred - label[i])
    
    loss = loss / n
    
    print("epoch = {}, loss = {:.3f}".format(epoch, loss))

epoch = 0, loss = 45.178
epoch = 1, loss = 42.856
epoch = 2, loss = 42.358
epoch = 3, loss = 41.948
epoch = 4, loss = 41.595


可以看到，在同样的训练周期内，使用随机梯度下降得到的训练损失更低，即损失收敛得更快。这是因为参数的实际更新次数要多得多。

接下来我们用随机梯度下降得到的参数 `w` 和 `b` 进行测试：

In [9]:
data = test_dataset
label = test_label
n = len(data)

loss = 0

for i in range(n):
    pred = linear_model(data[i])   
    loss += np.abs(pred - label[i])
    print("Pred = {:.3f}, Real = {:.3f}".format(pred, label[i]))

loss = loss / n

print("Average loss = {:.3f}".format(loss))

Pred = 18.598, Real = 16.800
Pred = 16.334, Real = 22.400
Pred = 16.458, Real = 20.600
Pred = 16.556, Real = 23.900
Pred = 16.423, Real = 22.000
Pred = 16.492, Real = 11.900
Average loss = 4.920


可以看到，虽然我们在训练集上的损失远低于使用批梯度下降的损失，但测试时得到的损失反而略高于批梯度下降的测试结果。为什么会这样？

**抛开数据、模型和损失函数本身的因素不谈，背后的原因可能是：**
- 使用随机梯度下降时，考虑到噪声数据的存在，并不一定参数每次更新都是朝着模型整体最优化的方向；
- 若样本噪声较多，很容易陷入局部最优解而收敛到不理想的状态；
- 如果更新次数过多，还容易出现在训练数据过拟合的情况，导致泛化能力变差。

既然两种梯度下降策略各有千秋，因此有一种折衷的方式，即采用**小批量梯度下降法（Mini-Batch Gradient Descent）**：

- 我们设定一个 `batch_size` 值，将数据划分为多个 `batch`，每个 `batch` 的数据采取批梯度下降策略来更新参数；
- 设置合适的 `batch_size` 值，既可以避免出现内存爆掉的情况，也使得损失可能更加平滑地收敛；
- 当 `batch_size=1` 时，小批量梯度下降其实就等同于随机梯度下降

天元 MegEngine 的优化器 `Optimizer` 中实现的 `SGD` 优化策略，实际上就是对小批量梯度下降法逻辑的实现。

这种折衷方案的效果比“真”随机梯度下降要好得多，**因为可以利用向量化加速批数据的运算，而不是分别计算每个样本。**

在这里我们先卖个关子，把向量化的介绍放在更后面，因为我们的当务之急是：获取小批量数据（Mini batch data）。

### 采样器（Sampler）

MegEngine 可以帮助用户根据设定的 `batch_size` 对样本进行采样 —— **除了已知的 `epochs` 和 `lr`，新的超参数出现了！**

现在我们先试试水，尝试人为地将 `train_dataset` 和 `train_label` 以 `batch_size=100` 的情况，顺序地进行划分：

In [10]:
batch_size = 100
batch_num = len(train_dataset) / batch_size

batch_dataset = []
for i in range(int(batch_num)):
    start = i * batch_size
    end = (i + 1) * batch_size
    batch_dataset.append((train_dataset[start:end], train_label[start:end]))

for batch_data, batch_label in batch_dataset:
    print(batch_data)
    print(len(batch_data))
    break

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [1.2083e-01 0.0000e+00 2.8900e+00 ... 1.8000e+01 3.9690e+02 4.2100e+00]
 [8.1870e-02 0.0000e+00 2.8900e+00 ... 1.8000e+01 3.9353e+02 3.5700e+00]
 [6.8600e-02 0.0000e+00 2.8900e+00 ... 1.8000e+01 3.9690e+02 6.1900e+00]]
100


这种对训练数据的顺序采样可以使用 MegEngine 中的 `SequentialSampler` 实现：

In [11]:
from megengine.data import SequentialSampler

sequential_sampler = SequentialSampler(dataset=train_dataset, batch_size=100)

# SequentialSampler 每次返回的是顺序索引，而不是划分后的数据本身
for indices in sequential_sampler:
    print(indices[0:5], "...", indices[95:100], len(indices))

[0, 1, 2, 3, 4] ... [95, 96, 97, 98, 99] 100
[100, 101, 102, 103, 104] ... [195, 196, 197, 198, 199] 100
[200, 201, 202, 203, 204] ... [295, 296, 297, 298, 299] 100
[300, 301, 302, 303, 304] ... [395, 396, 397, 398, 399] 100
[400, 401, 402, 403, 404] ... [495, 496, 497, 498, 499] 100


- 人工实现的逻辑是直接对数据集进行了划分，而 `Sampler` 返回的是索引值，而不是真实数据
- 因此要得到划分后的批数据，还需要使用后面即将介绍到的 `Dataloader` 模块
- 对于样本容量不是 `batch_size` 整数倍的情况，MegEngine 也能进行很好的处理
- 提示：对于测试数据，通常使用 `RandomSampler` 进行序列的随机采样

## 利用 Dataset 封装一个数据集

`Dataset` 是 MegEngine 中表示数据集最原始的的抽象类，可被其它类继承，可阅读 `Dataset` 模块的文档了解更多的细节。

在 MegEngine 中，想要通过使用 `Dataloader` 按照指定的 `Sampler` 读入真实数据，同时还需要提供受支持的 `Dataset` 类型.

我们自定义的数据集类应该继承 `MapDataset` 或者 `StreamDataset` 类，并重写下列方法：
- `__init__()` ：一般在其中实现读取数据源文件的功能。也可以添加任何其它的必要功能；
- `__getitem__()` ：通过索引操作来获取数据集中某一个样本，使得可以通过 for 循环来遍历整个数据集；
- `__len__()` ：返回数据集大小

根据波士顿房价数据集的特点，应该将它封装成 `MapDataset`（而 `StreamDataset` 常用于处理音频和视频等流数据）:

In [12]:
from typing import Tuple
from megengine.data.dataset import MapDataset

class BostonTrainDataset(MapDataset):
    def __init__(self):
        super().__init__()

        self.data = train_dataset
        self.label = train_label
    
    def __getitem__(self, index: int) -> Tuple:
        return self.data[index], self.label[index]

    def __len__(self) -> int:
        return len(self.data)
    
boston_train_dataset = BostonTrainDataset()
print(len(boston_train_dataset))

500


其实，对于这种单个或多个 NumPy 数组构成的数据，在 MegEngine 中也可以使用 `ArrayDataset` 对它进行初始化，它将自动完成以上方法的重写：

In [13]:
from megengine.data.dataset import ArrayDataset

boston_train_dataset = ArrayDataset(train_dataset, train_label)
print(len(boston_train_dataset))

500


### 数据载入器（Dataloader）

接下来便可以通过 `Dataloader` 来生成批数据，每个元素由对应的 `batch_data` 和 `batch_label` 组成：

In [14]:
from megengine.data import DataLoader

train_dataloader = DataLoader(dataset=boston_train_dataset, sampler=sequential_sampler)
for batch_data, batch_label in train_dataloader:
    print(batch_data)
    print(len(batch_data))
    break

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [1.2083e-01 0.0000e+00 2.8900e+00 ... 1.8000e+01 3.9690e+02 4.2100e+00]
 [8.1870e-02 0.0000e+00 2.8900e+00 ... 1.8000e+01 3.9353e+02 3.5700e+00]
 [6.8600e-02 0.0000e+00 2.8900e+00 ... 1.8000e+01 3.9690e+02 6.1900e+00]]
100


接下来我们一起来看看，使用批数据为什么能够加速整体的运算效率。

## 通过向量化加速运算

经过各种优化，在 NumPy 内部，向量化运算的速度是优于 `for` 循环的：

In [15]:
import time

n = 1000000
a = np.random.rand(n)
b = np.random.rand(n)

time_start = time.time()
c = np.dot(a, b)
time_end = time.time()
print('Vectorized version:', str(1000 * (time_end - time_start)), 'ms')

c = np.zeros(n)
time_start = time.time()
for i in range(n):
    c[i] = a[i] * b[i]
time_end = time.time()
print('For loop version:', str(1000 * (time_end - time_start)), 'ms')

Vectorized version: 0.4513263702392578 ms
For loop version: 563.164234161377 ms


重新阅读模型训练的代码，不难发现，每个 `epoch` 内部存在着 `for` 循环，根据模型定义的 $y_i = \boldsymbol {w} \cdot \boldsymbol {x_i} + b$ 进行了 $n$ 次计算。

我们在前面已经将数据集表示成了形状为 $(n, d)$ 的数据矩阵 $X$, 将标签表示成了 $y$ 向量，这启发我们利用矩阵的形式，一次性完成所有样本的前向计算过程：

$$
(y_{1}, y_{2}, \ldots, y_{d}) = 
\begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,d} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,d} \\
\vdots  &         &        & \vdots \\
x_{n,1} & x_{n,2} & \cdots & x_{n,d} 
\end{bmatrix}
\cdot (w_{1}, w_{2}, \ldots, w_{d}) + 
b
$$

我们设计一套测试样例来检验一下 `np.dot()` 这样的写法是否有效：

In [16]:
X = np.random.random((3, 2))
w = np.random.random((2))  
b = np.random.random(())
y = np.zeros((3, ))

for i in range(3):
    y[i] = np.dot(X[i], w) + b
    
Y = np.dot(X, w) + b

print(y)
print(Y)

[1.15814411 1.55340926 1.45122577]
[1.15814411 1.55340926 1.45122577]


接下来测试向量化是否起到了加速效果：

In [17]:
n = 50000
d = 1000

X = np.random.random((n, d))
w = np.random.random((d))  
b = np.random.random(())
y = np.zeros((n, ))

time_start = time.time()
Y = np.dot(X, w) + b
time_end = time.time()
print('Vectorized version:', str(1000 * (time_end - time_start)), 'ms')

time_start = time.time()
for i in range(n):
    y[i] = np.dot(X[i], w) + b
time_end = time.time()
print('For loop version:', str(1000 * (time_end - time_start)), 'ms')

Vectorized version: 12.490272521972656 ms
For loop version: 132.10368156433105 ms


可以发现后者运算速度明显快于前者。实际上，由于 NumPy 的广播机制，上面的 `dot()` 方法已经变成类似矩阵的运算：
$$
(y_{1}, y_{2}, \ldots, y_{d}) = 
\begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,d} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,d} \\
\vdots  &         &        & \vdots \\
x_{n,1} & x_{n,2} & \cdots & x_{n,d} 
\end{bmatrix}
\begin{bmatrix}
w_1 \\
w_2 \\
\vdots \\
w_d
\end{bmatrix} + 
b
$$

- 形状为 $(n,d)$ 的矩阵 $X$ 和维度为 $(d,)$ 的向量 $w$ 进行点乘，此时 $w$ 广播成形状为 $(d, 1)$ 的矩阵
- 两个矩阵进行乘法运算，此时的 `dot()` 等效于 `matmul()`, 底层效率比 `for` 循环快许多，得到形状为 $(n, 1)$ 的矩阵后，再变回维度为 $(n,)$ 的向量
- 最后完成 $(n,)$ 维向量元素间加法操作，得到最终想要的 $y$

### NumPy 实现

我们先尝试实现一下向量化的批梯度下降的代码：

In [18]:
epochs = 5
lr = 1e-6

w = np.zeros((13,))  
b = np.zeros(())

data = train_dataset
label = train_label

def linear_model(x):
    return np.dot(x, w) + b

for epoch in range(epochs):

    pred = linear_model(data)
    loss = np.sum((pred - label) ** 2)
    
    # 对应地，反向传播计算梯度的代码也需要改成向量化形式，这里不解释矩阵求导的推导过程
    w -= lr * 2 * np.dot(data.T, (pred - label)) / len(data)
    b -= lr * 2 * np.mean(pred - label)
    
    print("epoch = {}, loss = {:.3f}".format(epoch, loss / len(data)))

epoch = 0, loss = 594.442
epoch = 1, loss = 197.173
epoch = 2, loss = 138.077
epoch = 3, loss = 126.176
epoch = 4, loss = 121.148


上面的 `loss` 收敛状况与我们最开始实现的 `sum_grad` 的数值一致，你可以自己测试一下 `for` 循环写法和向量化写法的用时差异。

同时我们也可以发现，当前向传播使用批数据变成矩阵计算带来极大加速的同时，也为我们的反向传播梯度计算提出了更高的要求：
- 尽管存在着链式法则，但是对于不熟悉矩阵求导的人来说，还是很难理解一些梯度公式推导的过程，比如为什么出现了转置 `data.T` 
- 当前向传播的计算变得更加复杂，自动求导 `autograder` 机制就显得更加必要和方便

### MegEngine 实现

最后，让我们利用上小批量（Mini-batch）的数据把完整的训练流程代码在 MegEngine 中实现：
- 由于在 MegEngine 中 `F.dot()` 逻辑和 NumPy 的 `np.dot()` 并不完全一致，我们使用 `F.matmul()` 来代替
- 对应地，我们的 `w` 将由向量变为矩阵，`b` 在进行加法操作时将自动进行广播，变成矩阵
- 不同库或框架中同名方法的实现逻辑可以不同，这也提醒我们要及时阅读文档说明，或尝试阅读源码

In [19]:
import numpy as np
from sklearn.datasets import load_boston
import megengine as mge
from megengine.data.dataset import ArrayDataset
from megengine.data import SequentialSampler, RandomSampler, DataLoader
from megengine import functional as F
from megengine.autodiff import GradManager
import megengine.optimizer as optim

# 设置超参数
bs = 100
lr = 1e-6
epochs = 5
mask = 500

# 读取原始数据集
boston = load_boston()
X, y = boston.data, boston.target
total_num, num_features = boston.data.shape

# 训练数据加载与预处理
boston_train_dataset = ArrayDataset(X[:mask], y[:mask])
train_sampler = SequentialSampler(dataset=boston_train_dataset, batch_size=bs)
train_dataloader = DataLoader(dataset=boston_train_dataset, sampler=train_sampler)

# 初始化参数
w = mge.Parameter(np.zeros((num_features, 1)))
b = mge.Parameter(np.zeros(()))

# 定义模型
def linear_model(x):
    return F.matmul(x, w) + b

# 定义求导器和优化器
gm = GradManager().attach([w, b])
optimizer = optim.SGD([w, b], lr=lr)

# 模型训练
for epoch in range(epochs):
    total_loss = 0
    for batch_data, batch_label in train_dataloader:
        batch_data = mge.tensor(batch_data)
        batch_label = mge.tensor(batch_label)
        with gm:
            pred = linear_model(batch_data)
            loss = F.loss.square_loss(pred, batch_label)
            gm.backward(loss)
        optimizer.step().clear_grad()
        total_loss +=  loss.numpy().item()
    print("epoch = {}, loss = {:.3f}".format(epoch, total_loss / len(train_dataloader)))

epoch = 0, loss = 261.126
epoch = 1, loss = 135.167
epoch = 2, loss = 122.774
epoch = 3, loss = 115.320
epoch = 4, loss = 110.772


对我们训练好的模型进行测试：

In [20]:
boston_test_dataset = ArrayDataset(X[mask:num], y[mask:num])
test_sampler = RandomSampler(dataset=boston_test_dataset, batch_size=1)
test_dataloader = DataLoader(dataset=boston_test_dataset, sampler=test_sampler)

loss = 0
for batch_idx, (batch_data, batch_label) in enumerate(test_dataloader):
    batch_data = mge.tensor(batch_data)
    batch_label = mge.tensor(batch_label)
    pred = linear_model(batch_data)
    print("Pred = {:.3f}, Real = {:.3f}".format(pred[0].item(), batch_label.item()))
    loss += np.abs(pred[0].item() - batch_label.item())
print("Average loss = {:.3f}".format(loss / len(test_dataloader)))

Pred = 19.733, Real = 16.800
Pred = 19.195, Real = 11.900
Pred = 18.916, Real = 22.400
Pred = 19.091, Real = 22.000
Pred = 19.173, Real = 20.600
Pred = 19.251, Real = 23.900
Average loss = 3.783


可以发现，使用小批量梯度下降策略更新参数，最终在测试时得到的平均绝对值损失比单独采用批梯度下降和随机梯度下降策略时还要好一些。

## 总结回顾

有些时候，添加一些额外的条件，看似简单的问题就变得更加有趣起来了，就比如我们这次完成的稍微复杂那么一点儿的线性模型，想必你也收获满满吧？

我们需要及时总结学习过的知识，并在之后的时间内不断通过实践来强化记忆，这次的教程中出现了以下新的概念：
- 数据集（Dataset）：想要将现实中的数据集变成能够被 MegEngine 使用的格式，必须将其处理成 `MapDataset` 或者 `StreamDateset` 类
  - 继承基类的同时，还需要实现 `__init__()`, `__getitem__` 和 `__len__()` 三种内置方法
  - 我们对事物的抽象——特征（Feature）有了一定的认识，并接触了一些数学形式的表达如数据矩阵 $X$ 和标签向量 $y$ 等
  - 我们接触到了训练集（Trainining dataset）、测试集（Test dataset）和验证集（Validation dataset）/开发集（Develop dataset）的概念
  - 除了训练时我们会设计一个损失函数外，在测试模型性能时，我们也需要设计一个度量指标（Metric），不同任务的指标不尽相同
- 采样器（Sampler）：可以帮助我们自动地获得一个采样序列，常见的有顺序采样器 `SequentialSampler` 和随机采样器 `RandomSampler`
- 数据载入器（Dataloader）：根据 `Dataset` 和 `Sampler` 来获得对应的小批量数据（Mini-batch data）
  - 我们认识了一个新的超参数 `batch_size`，并且对不同形式的梯度下降各自的优缺点有了一定的认识
  - 我们理解了批数据向量化计算带来的好处，能够大大加快计算效率；同时为了避免推导向量化的梯度，实现自动求导机制是相当必要的
- **编程习惯：**及时查阅文档，比如 MegEngine 的 `F.dot()` 和 NumPy 的 `np.dot()` 背后的逻辑并不完全一致，直接替换将报错

## 问题思考

旧问题的解决往往伴随着新问题的诞生，让我们一起来思考一下：
- 我们在标记和收集数据的时候，特征的选取是否科学，数据集的规模应该要有多大？数据集的划分又应该选用什么样的比例？
- 这次见着了 `batch_size`, 它的设定是否有经验可循？接触到的超参数越来越多了，验证集和开发集是如何帮助我们对合适的超参数进行选取的？
- 我们对 NumPy 支持的 ndarray 的数据如何导入 MegEngine 已经有了经验，更复杂的数据集（图片、视频、音频）要如何处理呢？
- 噪声数据会对参数的学习进行干扰，`Dataloader` 中是否有对应的预处理方法来做一些统计意义上的处理？

深度学习，简单开发。我们鼓励你在实践中不断思考，并启发自己去探索直觉性或理论性的解释。

有了这么多的铺垫，是时候打开神经网络模型的大门了。